# Using Toolhouse for Tool Use with Groq API

[Toolhouse](https://app.toolhouse.ai/) is the first marketplace of AI tools, featuring tools like Code Interpreter, Web search and an Email tool. Rat . In this short demo we'll show how to use Toolhouse with Groq API, in particular Groq's finetuned for tool use `llama3-groq-70b-8192-tool-use-preview` model, for effective and fast tool calling.

### Setup

In [1]:
# Import packages
from toolhouse import Toolhouse
from groq import Groq

To use Groq and Toolhouse, set GROQ_API_KEY and TOOLHOUSE_API_KEY as environment variables:

In [2]:
client = Groq()
th = Toolhouse()

We will use the Groq custom finetuned [Llama3 70B Tool Use model](https://wow.groq.com/introducing-llama-3-groq-tool-use-models/). This model has been specifically finetuned off of Meta Llama3 70B for tool use and function calling, and achieves state-of-the-art performance on the Berkeley Function Calling Leaderboard:

In [3]:
MODEL = "llama3-groq-70b-8192-tool-use-preview"

We can use the `th.get_tools()` function to display all of the Toolhouse tools available:

In [4]:
print('TOOLS AVAILABLE:')
for tool in th.get_tools():
    print(f"Name: {tool['function']['name']}")
    print(f"Type: {tool['type']}")
    print(f"Description: {tool['function']['description']}")
    print('Required Parameters:')
    for required_parameter in tool['required']:
      print(f"{required_parameter} - {tool['function']['parameters']['properties'][required_parameter]['description']}")
    print('\n')

TOOLS AVAILABLE:
Name: code_interpreter
Type: function
Description: Allows you to run the code you generate. You can use this tool to verify that the code you generate is valid and contains all the relevant dependencies. IMPORTANT: When sending code, end your code with a print statement of the result. If you return something, make sure to change that return statement for a print statement.
Required Parameters:
code_str - The code to execute. Only Python is supported at the moment. IMPORTANT: When sending code, end your code with a print statement of the result. If you return something, make sure to change that return statement for a print statement.




For this demo, we only have one tool available: `code_interpretor`. This tool takes in the `code_str` parameter, which it identifies from the user message, and runs the code. 

### Configure Tool Call

First we'll configure the user message to the LLM with the code we want run:

In [5]:
# User message to the LLM
messages = [
  {
    "role": "user",
    "content": """Run this Python code: ```python
a = 409830948
print(a / 9834294824)
```""",
  }
]

We'll send this to our LLM via the Groq API. Note that this works just like a typical [Tool Use example in Groq](https://github.com/groq/groq-api-cookbook/blob/main/function-calling-101-ecommerce/Function-Calling-101-Ecommerce.ipynb), but instead of defining the tools ourselves we are using our existing tools in Toolhouse:

In [6]:
# Groq API response with Toolhouse tools
response = client.chat.completions.create(
  model=MODEL,
  messages=messages,
  # Passes Code Execution as a tool
  tools=th.get_tools(),
)

As you can see, the LLM properly identified that we'd like to invoke the code_interpreter tool, and properly passed our code to it via the `code_str` parameter:

In [7]:
tools_called = response.choices[0].message.tool_calls
for tool_called in tools_called:
    print(f"ID: {tool_called.id}")
    print(f"Type: {tool_called.type}")
    print(f"Function: {tool_called.function}")
    print('\n')

ID: call_2s9a
Type: function
Function: Function(arguments='{"code_str": "a = 409830948\\nprint(a / 9834294824)"}', name='code_interpreter')




### Execute Tool Call

Now we'll run the Code Execution tool, get the result, and append it to the context. The tool gets run through Toolhouse via the `run_tools` command, with the parameters that were identified in the previous LLM call.

As you can see from this tool run, our messages list now has entries for the assistant (the tool call configuration) and the tool (the tool call result):

In [8]:
tool_run = th.run_tools(response)
messages.extend(tool_run)

for message in tool_run:
    print(f"Role: {message['role']}")
    if 'tool_calls' in message:
        print(f"Tool Calls: {message['tool_calls']}")
    if 'tool_call_id' in message:
        print(f"Tool Call ID: {message['tool_call_id']}")
    print(f"Content: {message['content']}")
    print('\n')

Role: assistant
Tool Calls: [{'id': 'call_2s9a', 'function': {'arguments': '{"code_str": "a = 409830948\\nprint(a / 9834294824)"}', 'name': 'code_interpreter'}, 'type': 'function'}]
Content: None


Role: tool
Tool Call ID: call_2s9a
Content: 0.04167364872973225





We can compare the tool call result to the actual result by running the python code:

In [9]:
a = 409830948
print('ACTUAL RESULT:', a / 9834294824)

ACTUAL RESULT: 0.04167364872973225


Finally, we'll send our message list with the completed tool call back to the LLM to get a proper response:

In [10]:
response = client.chat.completions.create(
  model=MODEL,
  messages=messages,
  tools=th.get_tools(),
)

print('LLM RESPONSE:', response.choices[0].message.content)

LLM RESPONSE: The result of the division is 0.04167364872973225.
